Welcome!  If you are new to Google Colab/Jupyter notebooks, you might take a look at [this notebook](https://colab.research.google.com/notebooks/basic_features_overview.ipynb) first.

**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](http://underactuated.csail.mit.edu/simle_legs.html).**

# Notebook Setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- import packages used throughout the notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast (even on Colab) because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake (and underactuated).
if 'google.colab' in sys.modules and importlib.util.find_spec('underactuated') is None:
    urlretrieve(f"http://underactuated.csail.mit.edu/scripts/setup/setup_underactuated_colab.py",
                "setup_underactuated_colab.py")
    from setup_underactuated_colab import setup_underactuated
    setup_underactuated(underactuated_sha='15cfd96b0bdfd1b0c67597c24f91907776c02a6d', drake_version='0.27.0', drake_build='release')

server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

import numpy as np
from underactuated.jupyter import running_as_notebook

# The Rimless Wheel

In [ ]:
from pydrake.all import (DiagramBuilder, ConnectPlanarSceneGraphVisualizer,
                         ConnectMeshcatVisualizer,
                         SceneGraph, Simulator)
from pydrake.examples.rimless_wheel import (RimlessWheel, RimlessWheelGeometry,
                                            RimlessWheelParams)

def rimless_wheel(slope=0.08, initial_angle=0, initial_angular_velocity=5.0):
  params = RimlessWheelParams()
  params.set_slope(slope)

  builder = DiagramBuilder()
  rimless_wheel = builder.AddSystem(RimlessWheel())
  scene_graph = builder.AddSystem(SceneGraph())
  RimlessWheelGeometry.AddToBuilder(
      builder, rimless_wheel.get_floating_base_state_output_port(), params,
      scene_graph)
  visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url)
  visualizer.set_planar_viewpoint(xmin=-2, xmax=14, ymin=-2, ymax=3)

  diagram = builder.Build()
  simulator = Simulator(diagram)

  context = simulator.get_mutable_context()

  diagram.GetMutableSubsystemContext(
      rimless_wheel, context).get_numeric_parameter(0).set_slope(slope)
  context.SetAccuracy(1e-4)
  context.SetContinuousState([initial_angle, initial_angular_velocity])
  simulator.Initialize()
  simulator.set_target_realtime_rate(1.0)
  simulator.AdvanceTo(5.0 if running_as_notebook else 0.1)

rimless_wheel()

In [ ]:
rimless_wheel(initial_angular_velocity=5.0)

In [ ]:
rimless_wheel(initial_angular_velocity=10.0)

In [ ]:
rimless_wheel(initial_angular_velocity=0.95)

In [ ]:
rimless_wheel(initial_angular_velocity=-5.0)

In [ ]:
rimless_wheel(initial_angular_velocity=-4.8)

Here is a little interactive plot to allow you to visualize the trajectories of the rimless wheel as you vary the initial velocity.

TODO: Need to update this to make it work on colab.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

from underactuated.jupyter import SetupMatplotlibBackend
plt_is_interactive = SetupMatplotlibBackend()

from pydrake.all import Simulator
from pydrake.examples.rimless_wheel import RimlessWheel

rimless_wheel = RimlessWheel()
simulator = Simulator(rimless_wheel)
context = simulator.get_mutable_context()
params = context.get_numeric_parameter(0)
qmin = params.slope() - rimless_wheel.calc_alpha(params) - 0.1
qmax = params.slope() + rimless_wheel.calc_alpha(params) + 0.1
qdmin = -2
qdmax = 2

context.SetAccuracy(1e-2)
integrator = simulator.get_mutable_integrator()

fig, ax = plt.subplots()

# TODO(russt): make the slope interactive, too.
def simulate(initial_angular_velocity=1.5, duration=1.5):
    rimless_wheel.SetDefaultContext(context)
    context.SetTime(0.0)
    if initial_angular_velocity >= 0:
        initial_angle = params.slope() - rimless_wheel.calc_alpha(params)
    else:
        initial_angle = params.slope() + rimless_wheel.calc_alpha(params)
    if initial_angular_velocity == 0:
        # Set double_support = True.
        context.get_mutable_abstract_state(0).set_value(True)
        
    context.SetContinuousState([initial_angle, initial_angular_velocity])

    integrator.StartDenseIntegration()
    simulator.Initialize()
    simulator.AdvanceTo(duration)
    pp = integrator.StopDenseIntegration()

    return pp.vector_values(pp.get_segment_times())

if plt_is_interactive:
    data = simulate()
    line, = ax.plot(data[0,:], data[1,:],'b')
    pt, = ax.plot(data[0,0], data[1,0],'b*', markersize=12)
    
    def update(initial_angular_velocity):
        data = simulate(initial_angular_velocity)
        line.set_xdata(data[0,:])
        line.set_ydata(data[1,:])
        pt.set_xdata(data[0,0])
        pt.set_ydata(data[1,0])
        fig.canvas.draw()
    
    interact(update, initial_angular_velocity=widgets.FloatSlider(min=qdmin, max=qdmax, step=.1, value=1.1))

else:
    data = simulate()
    ax.plot(data[0,:], data[1,:],'b')
    ax.plot(data[0,0], data[1,0],'b*', markersize=12)

# Plot the energy contours.
nq = 151
nqd = 151
mgl = params.mass() * params.gravity() * params.length()
q = np.linspace(qmin, qmax, nq)
qd = np.linspace(qdmin, qdmax, nqd)
Q, QD = np.meshgrid(q, qd)
Energy = .5 * params.mass() * params.length()**2 * QD**2 + mgl * np.cos(Q)
ax.contour(Q,
           QD,
           Energy,
           alpha=0.5,
           linestyles="dashed",
           colors="black",
           linewidths=0.5)

ax.plot(params.slope() - rimless_wheel.calc_alpha(params)*np.array([1, 1]), np.array([0, qdmax]), 'k--')
ax.plot(params.slope() - rimless_wheel.calc_alpha(params)*np.array([1, 1]), np.array([0, qdmin]), 'k', linewidth=.25)
ax.plot(params.slope() + rimless_wheel.calc_alpha(params)*np.array([1, 1]), np.array([0, qdmin]), 'k--')
ax.plot(params.slope() + rimless_wheel.calc_alpha(params)*np.array([1, 1]), np.array([0, qdmax]), 'k', linewidth=.25)
ax.plot([qmin, qmax], [0, 0], 'k', linewidth=.25)
ax.plot([0, 0], [qdmin, qdmax], 'k', linewidth=.25)
ax.set_xlabel("theta")
ax.set_ylabel("thetadot")
ax.axis([qmin, qmax, qdmin, qdmax])
ax.set_title("Trajectories of the Rimless Wheel (w/ contours of "
             "constant energy)");

# The Compass Gait

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pydrake.all import (ConstantVectorSource, DiagramBuilder,
                         ConnectMeshcatVisualizer, SceneGraph, SignalLogger,
                         Simulator)
from pydrake.examples.compass_gait import (CompassGait, CompassGaitGeometry,
                                           CompassGaitParams)

def compass_gait():
    builder = DiagramBuilder()
    compass_gait = builder.AddSystem(CompassGait())

    hip_torque = builder.AddSystem(ConstantVectorSource([0.0]))
    builder.Connect(hip_torque.get_output_port(0), compass_gait.get_input_port(0))

    scene_graph = builder.AddSystem(SceneGraph())
    CompassGaitGeometry.AddToBuilder(
        builder, compass_gait.get_floating_base_state_output_port(), scene_graph)
    visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url)
    visualizer.set_planar_viewpoint(xmin=-1, xmax=5, ymin=-1, ymax=2)

    logger = builder.AddSystem(SignalLogger(14))
    builder.Connect(compass_gait.get_output_port(1), logger.get_input_port(0))

    diagram = builder.Build()
    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(1.0)

    context = simulator.get_mutable_context()
    context.SetAccuracy(1e-4)
    context.SetContinuousState([0., 0., 0.4, -2.])

    simulator.AdvanceTo(8.0)
    return logger

logger = compass_gait()

In [ ]:
plt.figure()
plt.plot(logger.data()[4, :], logger.data()[11, :])
plt.xlabel("left leg angle")
plt.ylabel("left leg angular velocity");